# This is a notebook for testing my implementation of LSPI

In [ ]:
# Messing around with OpenAI Gym

In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [49]:
env = gym.make('CartPole-v0')
#env = gym.make("NChain-v0")
env.reset()

array([ 0.00663644,  0.00141114, -0.04101978,  0.02104077])

# Implementation

In [184]:
def LSPI(basis_functions, gamma, epsilon, w):
    '''
    Compute the parameters of the policy, w, using the LSPI algorithm.
    
    Inputs:
    sample: list of tuples of the form (s,a,r,s')
    basis_functions: list of basis functions
    gamma: float, discount factor
    epsilon: float, convergence threshold
    w: intial policy parameter vector
    
    Outputs:
    w: the converged policy paramters
    '''
    
    while True:
        w_prev = w
        w = LSTDQ(basis_functions, gamma, w)
        
        if converged(w, w_prev, epsilon):
            break 
        else:
            w_prev = w
      
        print w
    return w

def converged(w, w_prev, epsilon):
    '''
    Determines if the policy parameters have converged based
    on whether or not the norm of the difference of w
    is less than the threshold epsilon.
    
    Inputs:
    w: a policy parameter vector
    w_prev: the policy parameter vetor from a previous iteration.
    epsilon: float, convergence threshold
    '''
    return np.linalg.norm(w - w_prev) < epsilon

def LSTDQ(basis_functions, gamma, w):
    '''
    Simple version of LSTDQ
    '''
    k = len(basis_functions)
    #A = np.zeros((k,k)), this might not have an inverse, use the next line instead
    A = np.identity(k) * 0.01
    b = np.zeros(k)
    
    sub_samples = generate_samples(100, 6)
    #samples[np.random.choice(len(samples), 100, replace=False)]
    
    for s, a, r, sp in sub_samples:
        phi = compute_phi(s,a, basis_functions)
        phi_p = compute_phi(sp, get_policy_action(sp, w, basis_functions), basis_functions)

        A += np.outer(phi, (phi - gamma*phi_p))
        b = b + phi*r
    
    
    w = np.dot(np.linalg.inv(A),b)
    return w
    
    

    
    
def LSTDQ_OPT(samples, basis_functions, gamma, w, sigma=0.1):
    '''
    Computes an approximation of the policy parameters based
    on the LSTDQ-OPT algorithm presented in the paper.
    
    Inputs:
    sample: list of tuples of the form (s,a,r,s')
    basis_functions: list of basis functions
    gamma: float, discount factor
    epsilon: float, convergence threshold
    w: intial policy parameter vector
    
     sigma: small positive float.
    '''
    pass
       

def compute_phi(s,a, basis_functions):
    '''
    Computes the vector ϕ(s,a) according to the basis function ϕ_1...ϕ_k
    
    Inputs:
    s: state
    a: action
    basis_functions: list of basis functions that operate on s and a
    
    Outputs:
    ϕ(s,a), a vector where each entry is the result of one of the basis functions.
    '''
    phi= np.array([bf(s,a) for bf in basis_functions])
    return phi
    
def get_policy_action(s, w, basis_functions):
    '''
    Given a parameterization for the policy,
    reconstruct the policy and querery it to get 
    the optimal action for state s. That is,
    the argmax over actions of ϕ(s,a).w
    
    Inputs:
    s: state
    w: policy parameters
    action_space: set of all possible actions
    
    Outputs:
    action a that the policy says is best
    '''
    a_max = None
    max_score = float("-inf")
    
    # TODO: don't hard code action space
    action_space = [0,1]
    
    # Search action space for most valuable action
    for a in action_space:
        #print "phi:", compute_phi(s,a, basis_functions)
        #print "w:",w
        score = np.dot(compute_phi(s,a, basis_functions), w)
        # update if we found something better
        if score > max_score:
            max_score = score
            a_max = a
    return a_max
    

def get_basis_functions(env, k):
    '''
    Define some basis functions and return them in a list
    '''
    bfs = []
    random_points = []

    s1 = env.observation_space.sample()
    s2 = env.observation_space.sample()
    s3 = env.observation_space.sample()
    s4 = env.observation_space.sample()
    
    s1 = np.array([1,1,1,1])
    s2 = np.array([0,0,0,0])
    s3 = np.array([-1,1,0,-1])
    
    
    print "s1:",s1
    print "s2:",s2
    bf1 = lambda s,a: 1
    bf2 = lambda s,a: np.exp( - np.linalg.norm(s-s1)/2.0)
    bf3 = lambda s,a: np.exp( - np.linalg.norm(s-s2)/2.0)
    bf4 = lambda s,a: np.exp( - np.linalg.norm(s-s3)/2.0)
    bf5 = lambda s,a: np.exp( - np.linalg.norm(s-s3)/2.0)
    bf6 = lambda s,a: int(a==0)*np.exp( - np.linalg.norm(s-s3)/2.0)
    bf7 = lambda s,a: int(a==1)*np.exp( - np.linalg.norm(s-s1)/2.0)
    
    bfs = [bf1,bf2,bf3,bf4, bf5, bf6, bf7]
    
    return bfs


def generate_samples(n_samples, n_steps=100):
    samples = []
    for i in range(n_samples):
        env.reset()
        for j in range(n_steps):
            s = list(env.env.state)
            a = env.action_space.sample()
            sp,r, _,_ = env.step(a)
            
            sample = (s, a, r, sp)
            samples.append(sample)

    return np.array(samples)
    

In [159]:
bfs[2](np.array([10,2,3,4]),10)

0.0

In [237]:
bfs = get_basis_functions(env,10)


gamma, epsilon, k = 0.1, 0.0005, len(bfs)
w = np.zeros(k)
w_est = LSPI(bfs, gamma, epsilon, w)
print w_est



s1: [1 1 1 1]
s2: [0 0 0 0]
[  1.09184325e+00   5.45928855e-02  -5.48213499e-03   4.85326940e-03
   4.85326940e-03   2.91067211e-04   6.13144830e-04]
[  1.09336421e+00   5.30367813e-02  -5.59175017e-03   3.92727159e-03
   3.92727159e-03   5.12810908e-04   4.70574722e-04]
[  1.09622980e+00   4.37699134e-02  -4.82974251e-03   3.71892722e-03
   3.71892722e-03   2.83828392e-04   3.65907299e-04]
[ 1.08983012  0.0602226  -0.00592895  0.00484164  0.00484164  0.00155095
  0.00189518]
[  1.09485345e+00   4.83544515e-02  -4.97532159e-03   3.97967432e-03
   3.97967432e-03  -3.76727524e-04  -5.13354676e-04]
[ 1.09189683  0.05495094 -0.00536673  0.004154    0.004154    0.00134502
  0.00134093]
[ 1.08920339  0.06123924 -0.00608415  0.00472265  0.00472265  0.0025849
  0.00318164]
[ 1.09200695  0.05436186 -0.00541566  0.00428234  0.00428234  0.00127351
  0.00170086]
[  1.09437070e+00   4.84282743e-02  -5.27254062e-03   4.15126316e-03
   4.15126316e-03   7.53686476e-04   8.64468150e-04]
[  1.09345760e+

[  1.09295874e+00   5.45753322e-02  -5.91619954e-03   4.12278974e-03
   4.12278973e-03   3.76459389e-04   4.16330017e-04]
[  1.09431540e+00   4.92831664e-02  -5.21175499e-03   4.28361817e-03
   4.28361818e-03  -1.91957392e-04  -1.43629510e-04]
[  1.09170518e+00   5.48926168e-02  -5.55554566e-03   5.44916164e-03
   5.44916165e-03  -4.59571684e-04  -2.01483664e-04]
[  1.09577605e+00   4.53298675e-02  -5.08073238e-03   3.89694725e-03
   3.89694726e-03   3.31420424e-04   4.44753064e-04]
[  1.09113501e+00   5.57321512e-02  -5.54864982e-03   5.32914268e-03
   5.32914267e-03   1.79012215e-04   5.34045244e-04]
[  1.09254893e+00   5.49103652e-02  -5.77788576e-03   4.26495105e-03
   4.26495105e-03   3.86905853e-04   3.75631510e-04]
[  1.08883715e+00   6.18693910e-02  -5.72896517e-03   5.44189976e-03
   5.44189976e-03   8.35821010e-04   9.90650224e-04]
[  1.09350515e+00   5.21304169e-02  -5.51479728e-03   4.03218888e-03
   4.03218888e-03   5.36352752e-04   6.95805795e-04]
[ 1.09454626  0.04912898

[  1.09397328e+00   5.03162718e-02  -5.44051163e-03   3.90311493e-03
   3.90311494e-03   9.78878107e-04   1.26161091e-03]
[ 1.09028591  0.05986704 -0.00567032  0.00421605  0.00421605  0.00143935
  0.00160274]
[  1.09178186e+00   5.50252480e-02  -5.46342177e-03   4.63173237e-03
   4.63173237e-03   7.28854767e-04   9.13649335e-04]
[  1.09346693e+00   5.30179525e-02  -5.91852955e-03   3.84857564e-03
   3.84857564e-03   1.04373001e-03   1.10811652e-03]
[  1.09523622e+00   4.80077423e-02  -5.36621602e-03   3.48111054e-03
   3.48111054e-03   8.16254924e-04   8.48294554e-04]
[ 1.09359533  0.05041667 -0.00561302  0.00397438  0.00397438  0.00193025
  0.00226662]
[  1.09492478e+00   4.86541948e-02  -5.46164503e-03   3.94834114e-03
   3.94834114e-03   2.09867822e-04   1.70877121e-04]
[  1.09595230e+00   4.55929360e-02  -5.09984189e-03   3.45952227e-03
   3.45952228e-03   5.45608025e-04   5.33874222e-04]
[ 1.08802061  0.06461707 -0.00649285  0.00517842  0.00517842  0.00237584
  0.00294912]
[  1.09

In [239]:
for i_episode in range(20):
    observation = env.reset()
    print "--------"
    for t in range(200):
        env.render()
        action = get_policy_action(env.env.state, w_est, bfs)
        #action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
#         if done:
#             print "reward:",reward
#             print("Episode finished after {} timesteps".format(t+1))
#             break

--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------
--------


In [176]:
env.reset()

array([ 0.02831727,  0.03855601, -0.02611418, -0.04275159])